In [1]:
from torch_geometric.nn.conv import MessagePassing

In [2]:
MessagePassing

torch_geometric.nn.conv.message_passing.MessagePassing

In [3]:
BATCH_SIZE = 32

In [4]:

from torch_geometric.loader import DataLoader
from dataset import SatDataset

test_path = r"C:\Users\leobo\Desktop\École\Poly\Recherche\Generic-Graph-Representation\Graph-Representation\src\models\sat\generic_sat\data\train"
dataset = SatDataset(root=test_path, graph_type="refactored", meta_connected_to_all=True, use_sat_label_as_feature=False)
loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [9]:
from torch.nn import Dropout
from torch_geometric.nn.models import MLP
import torch
import torch.nn.functional as F
from torch.nn import Linear
from mlp import LayerNormLSTMCell

In [11]:
dataset[0]


HeteroData(
  variable={
    x=[3, 1],
    y=[1]
  },
  value={ x=[2, 1] },
  operator={ x=[3, 1] },
  constraint={ x=[19, 2] },
  meta={ x=[1, 2] },
  (variable, has_domain, value)={ edge_index=[2, 6] },
  (variable, affected_by, operator)={ edge_index=[2, 3] },
  (variable, appears_in, constraint)={ edge_index=[2, 26] },
  (operator, connects_variable_to, constraint)={ edge_index=[2, 30] },
  (meta, has_constraint, constraint)={ edge_index=[2, 19] },
  (meta, has_variable, variable)={ edge_index=[2, 3] },
  (meta, has_value, value)={ edge_index=[2, 2] },
  (meta, has_operator, operator)={ edge_index=[2, 3] },
  (value, rev_has_domain, variable)={ edge_index=[2, 6] },
  (operator, rev_affected_by, variable)={ edge_index=[2, 3] },
  (constraint, rev_appears_in, variable)={ edge_index=[2, 26] },
  (constraint, rev_connects_variable_to, operator)={ edge_index=[2, 30] },
  (constraint, rev_has_constraint, meta)={ edge_index=[2, 19] },
  (variable, rev_has_variable, meta)={ edge_index=[2, 

In [10]:
class NeuroSatLSTM(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_layers, data, num_updates=26, dropout_prob=0, device="cuda:0"):
        self.hidden_channels = hidden_channels
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

        self.lstm_state_tuple = namedtuple('LSTMState', ('h','c'))
        self.final_reducer = F.relu
        self.decode_transfer_fn = F.relu

        self.variable_init = torch.empty((1, hidden_channels),device=self.device)
        torch.nn.init.xavier_uniform_(self.variable_init)
        
        # self.value_init = torch.empty((1, hidden_channels),device=self.device)
        # torch.nn.init.normal_(self.variable_init)
        
        self.operator_init = torch.empty((1, hidden_channels),device=self.device)
        torch.nn.init.xavier_uniform_(self.operator_init)
        
        self.constraint_init = torch.empty((1, hidden_channels),device=self.device)
        torch.nn.init.xavier_uniform_(self.constraint_init)

        self.value_msg = MLP(hidden_channels, repeat_end(hidden_channels, num_layers, hidden_channels), device=self.device) # 1 for variable
        self.variable_msg = MLP(hidden_channels * 3, repeat_end(hidden_channels, num_layers, hidden_channels), device=self.device) # 3 for constraint, operator, value
        self.operator_msg = MLP(hidden_channels * 2, repeat_end(hidden_channels, num_layers, hidden_channels), device=self.device) # 2 for constraint, variable
        self.constraint_msg = MLP(hidden_channels * 2, repeat_end(hidden_channels, num_layers, hidden_channels), device=self.device) # 2 for operator, variable

        self.variable_update = LayerNormLSTMCell(hidden_channels, activation=self.decode_transfer_fn, state_tuple=self.lstm_state_tuple, device=self.device)
        self.constraint_update = LayerNormLSTMCell(hidden_channels, activation=self.decode_transfer_fn, state_tuple=self.lstm_state_tuple, device=self.device)
        self.operator_update = LayerNormLSTMCell(hidden_channels, activation=self.decode_transfer_fn, state_tuple=self.lstm_state_tuple, device=self.device)
        self.value_update = LayerNormLSTMCell(hidden_channels, activation=self.decode_transfer_fn, state_tuple=self.lstm_state_tuple, device=self.device)

        self.variable_vote = MLP(hidden_channels, repeat_end(hidden_channels, num_layers, 1),device=self.device)
        self.vote_bias = torch.nn.Parameter(torch.zeros(1, device=self.device))    

        self.param_list = list(self.value_msg.parameters()) \
            + list(self.variable_msg.parameters()) \
            + list(self.operator_msg.parameters()) \
            + list(self.constraint_msg.parameters()) \
            + list(self.value_update.parameters()) \
            + list(self.variable_update.parameters()) \
            + list(self.operator_update.parameters()) \
            + list(self.constraint_update.parameters()) \
            + list(self.variable_vote.parameters()) + [self.vote_bias]
    
    def forward(self, node_dict, edge_dict, batch_dict):    
        value_output = torch.tile(self.value_init, [2, 1])
        variable_output = torch.tile(self.variable_init, [self.num_variables, 1])
        constraint_output = torch.tile(self.operator_init, [self.num_constraints, 1])
        operator_output = torch.tile(self.constraint_init, [self.num_operators, 1])

        value_state = self.lstm_state_tuple(h=value_output, c=torch.zeros([2, self.hidden_channels],device=self.device))
        variable_state = self.lstm_state_tuple(h=variable_output, c=torch.zeros([self.num_variables, self.hidden_channels],device=self.device))
        operator_state = self.lstm_state_tuple(h=operator_output, c=torch.zeros([self.num_operators, self.hidden_channels],device=self.device))
        constraint_state = self.lstm_state_tuple(h=constraint_output, c=torch.zeros([self.num_constraints, self.hidden_channels],device=self.device))

        for _ in range(self.num_updates):
            value_pre_msgs = self.value_msg.forward(variable_state.h)
            value_state = self.value_update(inputs=value_pre_msgs, state=value_state)
            
            variable_input = torch.cat([value_state.h, operator_state.h, constraint_state.h], axis=1)
            variable_pre_msgs = self.variable_msg.forward(variable_input)
            variable_state = self.variable_update(inputs=variable_pre_msgs, state=variable_state)
            
            operator_input = torch.cat([variable_state.h, constraint_state.h], axis=1)
            operator_pre_msgs = self.operator_msg.forward(operator_input)
            operator_state = self.operator_update(inputs=operator_pre_msgs, state=operator_state)

            constraint_input = torch.cat([variable_state.h, operator_state.h], axis=1)
            constraint_pre_msgs = self.constraint_msg.forward(constraint_input)
            constraint_state = self.constraint_update(inputs=constraint_pre_msgs, state=constraint_state)
        
        return variable_state.h
        

In [13]:
sample = dataset[0]

In [14]:
sample

HeteroData(
  variable={
    x=[3, 1],
    y=[1]
  },
  value={ x=[2, 1] },
  operator={ x=[3, 1] },
  constraint={ x=[19, 2] },
  meta={ x=[1, 2] },
  (variable, has_domain, value)={ edge_index=[2, 6] },
  (variable, affected_by, operator)={ edge_index=[2, 3] },
  (variable, appears_in, constraint)={ edge_index=[2, 26] },
  (operator, connects_variable_to, constraint)={ edge_index=[2, 30] },
  (meta, has_constraint, constraint)={ edge_index=[2, 19] },
  (meta, has_variable, variable)={ edge_index=[2, 3] },
  (meta, has_value, value)={ edge_index=[2, 2] },
  (meta, has_operator, operator)={ edge_index=[2, 3] },
  (value, rev_has_domain, variable)={ edge_index=[2, 6] },
  (operator, rev_affected_by, variable)={ edge_index=[2, 3] },
  (constraint, rev_appears_in, variable)={ edge_index=[2, 26] },
  (constraint, rev_connects_variable_to, operator)={ edge_index=[2, 30] },
  (constraint, rev_has_constraint, meta)={ edge_index=[2, 19] },
  (variable, rev_has_variable, meta)={ edge_index=[2, 

In [17]:
loader.get(0)

AttributeError: 'DataLoader' object has no attribute 'get'

In [18]:
sample.x_dict

{'variable': tensor([[1.],
         [1.],
         [1.]]),
 'value': tensor([[0.],
         [1.]]),
 'operator': tensor([[-1.],
         [-1.],
         [-1.]]),
 'constraint': tensor([[1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 0.6667],
         [1.0000, 1.0000],
         [1.0000, 1.0000],
         [1.0000, 1.0000]]),
 'meta': tensor([[19.,  3.]])}

In [24]:
dataset[1400].edge_index_dict

{('variable',
  'has_domain',
  'value'): tensor([[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6],
         [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]]),
 ('variable',
  'affected_by',
  'operator'): tensor([[6, 4, 1, 5, 0, 2, 3],
         [6, 4, 1, 5, 0, 2, 3]]),
 ('variable',
  'appears_in',
  'constraint'): tensor([[ 0,  2,  4,  6,  2,  3,  3,  6,  5,  3,  4,  6,  1,  5,  2,  2,  3,  5,
           4,  0,  1,  2,  3,  4,  5,  6,  6,  2,  3,  5,  0,  1,  4,  0,  4,  6,
           2,  1,  2,  3,  4,  5,  1,  2,  4,  2,  5,  3,  6,  5,  3,  6,  1,  2,
           2,  4,  3,  4,  0,  2,  1,  3,  4,  0,  1,  4,  2,  3,  4,  5,  2,  1,
           2,  5,  2,  5,  0,  1,  2,  3,  0,  4,  5,  0,  1,  3,  4,  5,  2,  4,
           4,  3,  5,  4,  5,  0],
         [ 1,  1,  1,  1,  2,  2,  3,  4,  5,  6,  6,  6,  8,  8, 10, 11, 11, 11,
          12, 13, 13, 13, 13, 13, 13, 13, 14, 15, 15, 15, 17, 17, 17, 18, 18, 18,
          19, 20, 20, 20, 20, 20, 22, 22, 22, 23, 24, 25, 25, 25, 26, 26, 27, 27,
   

In [30]:
variable_init = torch.empty((1, 3),device="cuda")
torch.nn.init.xavier_normal_(variable_init)

tensor([[0.3862, 0.8596, 0.7696]], device='cuda:0')

In [33]:
variable = torch.tile(variable_init, [7, 1])

In [34]:
variable

tensor([[0.3862, 0.8596, 0.7696],
        [0.3862, 0.8596, 0.7696],
        [0.3862, 0.8596, 0.7696],
        [0.3862, 0.8596, 0.7696],
        [0.3862, 0.8596, 0.7696],
        [0.3862, 0.8596, 0.7696],
        [0.3862, 0.8596, 0.7696]], device='cuda:0')

In [36]:
iter_loader = iter(loader)

In [37]:
batch = next(iter_loader)

In [40]:
batch.batch_dict

{'variable': tensor([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  5,
          6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 11, 11,
         12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17,
         18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23,
         24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29,
         30, 30, 30, 31, 31, 31]),
 'value': tensor([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,
          9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17,
         18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26,
         27, 27, 28, 28, 29, 29, 30, 30, 31, 31]),
 'operator': tensor([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  5,
          6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 11, 11,
         12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17

In [41]:
batch.edge_index_dict

{('variable',
  'has_domain',
  'value'): tensor([[ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,
           9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17,
          18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26,
          27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35,
          36, 36, 37, 37, 38, 38, 39, 39, 40, 40, 41, 41, 42, 42, 43, 43, 44, 44,
          45, 45, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50, 51, 51, 52, 52, 53, 53,
          54, 54, 55, 55, 56, 56, 57, 57, 58, 58, 59, 59, 60, 60, 61, 61, 62, 62,
          63, 63, 64, 64, 65, 65, 66, 66, 67, 67, 68, 68, 69, 69, 70, 70, 71, 71,
          72, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 78, 78, 79, 79, 80, 80,
          81, 81, 82, 82, 83, 83, 84, 84, 85, 85, 86, 86, 87, 87, 88, 88, 89, 89,
          90, 90, 91, 91, 92, 92, 93, 93, 94, 94, 95, 95],
         [ 0,  1,  0,  1,  0,  1,  2,  3,  2,  3,  2,  3,  4,  5,  4,  5,  4,  5

In [42]:
torch.sparse_coo_tensor

<function torch._VariableFunctionsClass.sparse_coo_tensor>

In [43]:
 batch

HeteroDataBatch(
  variable={
    x=[96, 1],
    y=[32],
    batch=[96],
    ptr=[33]
  },
  value={
    x=[64, 1],
    batch=[64],
    ptr=[33]
  },
  operator={
    x=[96, 1],
    batch=[96],
    ptr=[33]
  },
  constraint={
    x=[680, 2],
    batch=[680],
    ptr=[33]
  },
  meta={
    x=[32, 2],
    batch=[32],
    ptr=[33]
  },
  (variable, has_domain, value)={ edge_index=[2, 192] },
  (variable, affected_by, operator)={ edge_index=[2, 96] },
  (variable, appears_in, constraint)={ edge_index=[2, 952] },
  (operator, connects_variable_to, constraint)={ edge_index=[2, 996] },
  (meta, has_constraint, constraint)={ edge_index=[2, 680] },
  (meta, has_variable, variable)={ edge_index=[2, 96] },
  (meta, has_value, value)={ edge_index=[2, 64] },
  (meta, has_operator, operator)={ edge_index=[2, 96] },
  (value, rev_has_domain, variable)={ edge_index=[2, 192] },
  (operator, rev_affected_by, variable)={ edge_index=[2, 96] },
  (constraint, rev_appears_in, variable)={ edge_index=[2, 952

In [44]:
batch.x_dict

def get_num_variables_per_problem(data_batch):
    pass

In [47]:
len(batch.x_dict["variable"])

96

In [48]:
batch.batch_dict

{'variable': tensor([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  5,
          6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 11, 11,
         12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17,
         18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23,
         24, 24, 24, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28, 28, 29, 29, 29,
         30, 30, 30, 31, 31, 31]),
 'value': tensor([ 0,  0,  1,  1,  2,  2,  3,  3,  4,  4,  5,  5,  6,  6,  7,  7,  8,  8,
          9,  9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17,
         18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26,
         27, 27, 28, 28, 29, 29, 30, 30, 31, 31]),
 'operator': tensor([ 0,  0,  0,  1,  1,  1,  2,  2,  2,  3,  3,  3,  4,  4,  4,  5,  5,  5,
          6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 11, 11,
         12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17

In [65]:
from collections import Counter
counts = {
    "variable": None,
    "value": None,
    "operator": None,
    "constraint": None
}

for k in counts.keys():
    counts[k] = Counter(batch.batch_dict[k].numpy())

In [67]:
counts["variable"][1]

3

In [68]:
mlp = MLP([4,4,4])

In [72]:
mlp

MLP(4, 4, 4)

In [76]:
haha = torch.randn((2,4))
mlp(haha)

tensor([[ 0.0085,  0.5124,  0.3079, -0.0164],
        [ 0.2307, -0.0821,  0.5696,  0.3730]], grad_fn=<AddmmBackward0>)

In [84]:
# num_vars = 
torch.tile(variable_init, [list(num_vars.values()), 1])

TypeError: tile(): argument 'dims' (position 2) must be tuple of ints, not list

In [81]:
list(num_vars.values())

[3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3]

In [91]:
import pickle
# Copyright 2018 Daniel Selsam. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

import numpy as np
import math

# TODO(dhs): duplication
def ilit_to_var_sign(x):
    assert(abs(x) > 0)
    var = abs(x) - 1
    sign = x < 0
    return var, sign

# TODO(dhs): duplication
def ilit_to_vlit(x, n_vars):
    assert(x != 0)
    var, sign = ilit_to_var_sign(x)
    if sign: return var + n_vars
    else: return var

class Problem(object):
    def __init__(self, n_vars, iclauses, is_sat, n_cells_per_batch, all_dimacs):
        self.n_vars = n_vars
        self.n_lits = 2 * n_vars
        self.n_clauses = len(iclauses)

        self.n_cells = sum(n_cells_per_batch)
        self.n_cells_per_batch = n_cells_per_batch

        self.is_sat = is_sat
        self.compute_L_unpack(iclauses)

        # will be a list of None for training problems
        self.dimacs = all_dimacs

    def compute_L_unpack(self, iclauses):
        self.L_unpack_indices = np.zeros([self.n_cells, 2], dtype=int)
        cell = 0
        for clause_idx, iclause in enumerate(iclauses):
            vlits = [ilit_to_vlit(x, self.n_vars) for x in iclause]
            for vlit in vlits:
                self.L_unpack_indices[cell, :] = [vlit, clause_idx]
                cell += 1

        assert(cell == self.n_cells)

def shift_ilit(x, offset):
    assert(x != 0)
    if x > 0: return x + offset
    else:     return x - offset

def shift_iclauses(iclauses, offset):
    return [[shift_ilit(x, offset) for x in iclause] for iclause in iclauses]

def mk_batch_problem(problems):
    all_iclauses = []
    all_is_sat = []
    all_n_cells = []
    all_dimacs = []
    offset = 0

    prev_n_vars = None
    for dimacs, n_vars, iclauses, is_sat in problems:
        assert(prev_n_vars is None or n_vars == prev_n_vars)
        prev_n_vars = n_vars

        all_iclauses.extend(shift_iclauses(iclauses, offset))
        all_is_sat.append(is_sat)
        all_n_cells.append(sum([len(iclause) for iclause in iclauses]))
        all_dimacs.append(dimacs)
        offset += n_vars

    return Problem(offset, all_iclauses, all_is_sat, all_n_cells, all_dimacs)

filename = r"C:\Users\leobo\Desktop\École\Poly\Recherche\Generic-Graph-Representation\Graph-Representation\src\models\sat\neurosat\data\train\sr5\data_dir=grp1_npb=100_nb=1328.pkl"
with open(filename, 'rb') as f:
    problems = pickle.load(f)

In [90]:
import os
os.chdir(r"C:\Users\leobo\Desktop\École\Poly\Recherche\Generic-Graph-Representation\Graph-Representation\src\models\sat\neurosat\python")

In [93]:
import os
import pickle
import mk_problem

class ProblemsLoader(object):
    def __init__(self, filenames):
        self.filenames = filenames
        print(self.filenames)

        self.next_file_num = 0
        assert(self.has_next())

    def has_next(self):
        return self.next_file_num < len(self.filenames)

    def get_next(self):
        if not self.has_next():
            self.reset()
        filename = self.filenames[self.next_file_num]
        # print("Loading %s..." % filename)
        with open(filename, 'rb') as f:
            problems = pickle.load(f)
        self.next_file_num += 1
        assert(len(problems) > 0)
        return problems, filename

    def reset(self):
        self.next_file_num = 0

def init_problems_loader(dirname):
    return ProblemsLoader([dirname + "/" + f for f in os.listdir(dirname)])


In [96]:
dirname = r"C:\Users\leobo\Desktop\École\Poly\Recherche\Generic-Graph-Representation\Graph-Representation\src\models\sat\neurosat\data\train\sr5"
PL = init_problems_loader(dirname)

['C:\\Users\\leobo\\Desktop\\École\\Poly\\Recherche\\Generic-Graph-Representation\\Graph-Representation\\src\\models\\sat\\neurosat\\data\\train\\sr5/data_dir=grp1_npb=100_nb=1328.pkl', 'C:\\Users\\leobo\\Desktop\\École\\Poly\\Recherche\\Generic-Graph-Representation\\Graph-Representation\\src\\models\\sat\\neurosat\\data\\train\\sr5/data_dir=grp2_npb=100_nb=1366.pkl']


In [98]:
train_problems, train_filename = PL.get_next()

In [103]:
problem = train_problems[0]
d = {}
d['n_vars'] = torch.tensor(problem.n_vars,device="cuda")
d['n_lits'] = torch.tensor(problem.n_lits,device="cuda")
d['n_clauses'] = torch.tensor(problem.n_clauses,device="cuda")
d['L_unpack'] = torch.sparse_coo_tensor(indices=torch.tensor(problem.L_unpack_indices.T, dtype=torch.long),
                                   values=torch.tensor(np.ones(problem.L_unpack_indices.shape[0]), dtype=torch.float),
                                   size=(problem.n_lits, problem.n_clauses))
d['is_sat'] = torch.tensor(problem.is_sat,device="cuda")

In [104]:

d

{'n_vars': tensor(12, device='cuda:0'),
 'n_lits': tensor(24, device='cuda:0'),
 'n_clauses': tensor(76, device='cuda:0'),
 'L_unpack': tensor(indices=tensor([[14,  0, 12, 13,  2,  1,  2,  0,  1, 14,  1,  0,  2, 13,
                          2, 12,  1,  0, 14, 12, 13,  2, 14,  0, 14, 12, 13, 12,
                          2, 12,  2,  1, 14,  0,  1, 12,  2, 13,  1,  2,  0,  0,
                          2, 17,  3, 15, 16,  5,  4,  5,  3,  4, 17,  4,  3,  5,
                         16,  5, 15,  4,  3, 17, 15, 16,  5, 17,  3, 17, 15, 16,
                         15,  5, 15,  5,  4, 17,  3,  4, 15,  5, 16,  4,  5,  3,
                         15,  5, 18,  8,  7, 18,  7,  8, 20,  6,  7, 18,  7,  8,
                          6,  8, 19,  8,  6, 19, 19,  6,  8,  6, 19, 20, 20, 19,
                          7,  8, 18,  6,  7, 20, 19,  6,  8,  6, 19, 20,  8, 18,
                          7,  8,  6,  7, 20, 18,  6,  8, 19,  7,  6, 20,  7, 20,
                          6, 18,  8,  7,  6,  7, 20,  8

In [105]:
train_filename

'C:\\Users\\leobo\\Desktop\\École\\Poly\\Recherche\\Generic-Graph-Representation\\Graph-Representation\\src\\models\\sat\\neurosat\\data\\train\\sr5/data_dir=grp1_npb=100_nb=1328.pkl'